In [0]:
import numpy as np
import pandas as pd
import string 
from nltk.tokenize import TweetTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
train_ = pd.read_csv('/content/drive/My Drive/train.csv', delimiter="\t")
test_=pd.read_csv('/content/drive/My Drive/test.csv', delimiter="\t")
print(train_.head())
print(test_.head())

            meta  uid sentiment
0           meta    3  negative
1              @    O       NaN
2  AdilNisarButt  Hin       NaN
3       pakistan  Hin       NaN
4             ka  Hin       NaN
              meta  uid sentiment
0             meta    8   neutral
1               RT  Eng       NaN
2                @    O       NaN
3  UAAPconfessions  Eng       NaN
4             Love  Eng       NaN


In [0]:
def extractdat(dataframe):
  data = dataframe.loc[:,['meta','sentiment']]
  t = ""
  x = list()
  y = list()
  for i in range(data.shape[0]):
    if(data.iloc[i,0]=="meta"):
      x.append(t.strip())
      y.append(str(data.iloc[i,1]))
      t=""
    else:
      t += (str(data.iloc[i,0])+" ")
  
  x.append(t.strip())
  
  return x[1:], y

In [0]:
X_train, Y_train = extractdat(train_)
X_test, Y_test = extractdat(test_)

In [10]:
print(len(X_train), len(Y_train), len(X_test), len(Y_test))

13524 13524 1869 1869


In [0]:
def clean(text, tknzr):
  tweet_tokens = tknzr.tokenize(text)
  t = ""
  for j in tweet_tokens:
    if(j not in string.punctuation and j not in ['…']):
      t += (j.lower())
      t+=" "
  return t.strip()

In [0]:
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

In [0]:
for i in range(len(X_train)):
  X_train[i]=(clean(X_train[i],tknzr))

In [0]:
for i in range(len(X_test)):
  X_test[i]=(clean(X_test[i], tknzr))

In [0]:
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(X_train)

In [0]:
X_train = tk.texts_to_sequences(X_train)
X_test = tk.texts_to_sequences(X_test)
tweets_lengths = list()

In [17]:
X_train = pad_sequences(X_train, maxlen=200, padding='post')
X_test = pad_sequences(X_test, maxlen=200, padding='post')
X_train = np.array(X_train, dtype='float32')
X_test = np.array(X_test, dtype='float32')
from keras.utils import to_categorical
def categorize(arr):
  cat = list()
  for i in arr:
    if(i == "positive"):
      cat.append(0)
    elif(i == "negative"):
      cat.append(1)
    else:
      cat.append(2)
  cat = np.array(cat,dtype='float32')
  categorical = to_categorical(cat)
  return categorical

Y_train = categorize(Y_train)
Y_test = categorize(Y_test)
print(len(Y_test))
print(len(X_test))
vocab_size = len(tk.word_index)
print(vocab_size)
embedding_weights = []
for char, i in tk.word_index.items():
    onehot = np.zeros(vocab_size)
    onehot[i - 1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)


1869
1869
880


In [18]:
from keras.models import Sequential
from keras.layers import Dense, Input, Conv1D, GlobalMaxPool1D, Embedding, Flatten, Dropout, MaxPooling1D
from keras.layers import LSTM, Lambda, TimeDistributed, Bidirectional, concatenate
embedding_size = vocab_size
conv_layers = [[256, 7, 3], [256, 7, 3], [256, 3, -1],[256, 3, -1], [256, 3, -1],[256, 3, 3]]
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length = 200, weights=[embedding_weights]))
for i, j, k in conv_layers:
    model.add(Conv1D(i, j, activation='relu'))
    if k != -1:
        model.add(MaxPooling1D(pool_size=k))
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=3, activation='softmax'))
model.summary()
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 880)          774400    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 194, 256)          1577216   
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 64, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 58, 256)           459008    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 19, 256)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 17, 256)           19686

In [19]:
print(len(X_test),len(Y_test))

1869 1869


In [20]:
print(len(X_train),len(Y_train))

13524 13524


In [22]:
model.fit(X_train, Y_train, batch_size=128, epochs=30, validation_data=(X_test, Y_test), verbose=1)

Train on 13524 samples, validate on 1869 samples
Epoch 1/30
13524/13524 [==============================] - 14s 1ms/step - loss: 0.0524 - acc: 0.9818 - val_loss: 2.9990 - val_acc: 0.5206
Epoch 2/30
13524/13524 [==============================] - 14s 1ms/step - loss: 0.0623 - acc: 0.9786 - val_loss: 2.8169 - val_acc: 0.5051
Epoch 3/30
13524/13524 [==============================] - 14s 1ms/step - loss: 0.0524 - acc: 0.9835 - val_loss: 3.1189 - val_acc: 0.5104
Epoch 4/30
13524/13524 [==============================] - 14s 1ms/step - loss: 0.0419 - acc: 0.9850 - val_loss: 3.3451 - val_acc: 0.5174
Epoch 5/30
13524/13524 [==============================] - 14s 1ms/step - loss: 0.0397 - acc: 0.9854 - val_loss: 3.2127 - val_acc: 0.4933
Epoch 6/30
13524/13524 [==============================] - 14s 1ms/step - loss: 0.0412 - acc: 0.9845 - val_loss: 3.3380 - val_acc: 0.4906
Epoch 7/30
13524/13524 [==============================] - 14s 1ms/step - loss: 0.0620 - acc: 0.9784 - val_loss: 3.1918 - val_acc:

In [0]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score

In [0]:
f1score = []
recalls = []
precisions = []

predict = (np.array(model.predict(X_test))).round() 
targ = Y_test
f1 = f1_score(targ, predict,average=None)
recall = recall_score(targ, predict,average=None)
precision = precision_score(targ, predict,average=None)
accuracy = accuracy_score(targ, predict)
f1score.append(f1)
recalls.append(recall)
precisions.append(precision)


In [26]:
print("f1_Positive: %f " %(f1[0]))
print("f1_Negetive: %f " %(f1[1]))
print("f1_Neutral: %f " %(f1[2]))
print("precision_Positive: %f" %(precision[0]))
print("precision_Negetive: %f" %(precision[1]))
print("precision_Neutral: %f" %(precision[2]))
print("recall_Positive %f " %(recall[0]))
print("nrecall_Negetive %f" %(recall[1]))
print("recall_Neutral %f" %(recall[2]))
print("Accuracy: %f percent" %(accuracy*100))

f1_Positive: 0.555369 
f1_Negetive: 0.487047 
f1_Neutral: 0.507292 
precision_Positive: 0.542623
precision_Negetive: 0.543981
precision_Neutral: 0.486027
recall_Positive 0.568729 
nrecall_Negetive 0.440901
recall_Neutral 0.530504
Accuracy: 51.685393 percent
